<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MMLU_MISTRAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers
!pip install -q tqdm
!pip install -q pandas
!pip install -q tensor_parallel
!pip install -q argparse
!pip install -q einops
!pip install -q accelerate
#!pip install -q torch==2.0.0+cu118
!pip install -q torch

!pip install colab-env --upgrade -q
!pip install openai -q

!pip install datasets -q
!pip install utils -q

In [ ]:
!git clone https://github.com/FranxYao/chain-of-thought-hub.git

In [ ]:
import colab_env
import os
import openai
import IPython
import pytz
from datetime import datetime
import json
from pathlib import Path

In [4]:
def test_answer_mmlu_(pred_str, ans):
    pattern = 'the answer is ('
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1][0]
        gold = ans.lower()
        # print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'C'
        # print(ans_str)
        gold = ans.lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

# extract answer in pred_str and compare with ans_str
def test_answer_mmlu_claude_instant(pred_str, ans_str):
    pattern = 'the answer is '
    pred = pred_str.lower().split(pattern)
    if len(pred) == 1:
        return False
    else:
        return pred[1][0] == ans_str.lower()

def test_answer_mmlu_claude(pred_str, ans_str):
    pattern = 'the answer is '
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1]
        for p in pred:
            if(p.isalpha()): break
        pred = p
        print(ans_str)
        gold = ans_str.lower()
        print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'c'
        # print(ans_str)
        gold = ans_str.lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

def test_answer_mmlu(pred_str, ans_str):
    pattern = 'the answer is ('
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1][0]
        gold = ans_str.split('A:\n')[1][0].lower()
        # print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'C'
        # print(ans_str)
        gold = ans_str.split('A:\n')[1][0].lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                # print(am)
                # print(a)
                if(test_answer_mmlu(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:') and not l.startswith("A: Let's think step by step")):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)

    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    # print(am)
    # print(a)
    if(test_answer_mmlu(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [5]:
TASKSTEST0old = [
        'anatomy',
        'college_biology',
        'college_chemistry',
        'college_computer_science',
        'college_mathematics',
        'college_medicine',
        'college_physics',
        'computer_security',
        'electrical_engineering',
        'machine_learning',
]


TASKSTEST = [
        'college_computer_science',
        'electrical_engineering',
        'machine_learning',
]

TASKS911 = [
        'anatomy',
        'college_biology',
        'college_chemistry',
        'college_computer_science',
        'college_mathematics',
        'college_medicine',
        'college_physics',
        'computer_security',
        'conceptual_physics',
        'econometrics',
        'electrical_engineering',
        'elementary_mathematics',
        'formal_logic',
        'global_facts',
        'high_school_biology',
        'high_school_chemistry',
        'high_school_computer_science',
        'high_school_european_history',
        'high_school_geography',
        'high_school_government_and_politics',
        'high_school_macroeconomics',
        'high_school_mathematics',
        'high_school_microeconomics',
        'high_school_physics',
        'high_school_psychology',
        'high_school_statistics',
        'high_school_us_history',
        'high_school_world_history',
        'public_relations',
        'security_studies',
        'sociology',
        'us_foreign_policy',
        'virology',
        'machine_learning',
        'world_religions']

In [ ]:
!pip install mistralai --quiet

# Install Hugging Face libraries
!pip install  --upgrade bitsandbytes --quiet
from datasets import load_dataset

In [7]:
import mistralai
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os
import colab_env
import json

In [8]:
# datetime object containing current date and time
newYorkTz = pytz.timezone("America/New_York")
now = datetime.now(newYorkTz)
#print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
#print("date and time =", dt_string)

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"
client = MistralClient(api_key=api_key)

print()
print('TEST - MISTRAL  - BY FRANK MORALES - %s'%dt_string)
print()


TEST - MISTRAL  - BY FRANK MORALES - 01/05/2024 00:40:19



In [9]:
def mistral_chat(prompt,model,client,stream):
  client = MistralClient(api_key=api_key)
  messages = [ChatMessage(role="user", content=prompt)]
  if stream==True:
     print("Streaming")
     print()
     print('Answer: ')
     for chunk in client.chat_stream(
        model=model,
        messages=messages,
        ):
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
  else:

     #print("No streaming")
     chat_response = client.chat(model=model,messages=messages, safe_mode=True,temperature=0.9)
     #print(chat_response.choices[0].message.content)
     return chat_response

In [10]:
import json
model = "mistral-large-latest"
query = "What is the average grid for competitors with over 22 laps and time/retired of +17.276? Answer me as SQL and explain your solutions."
chat_response=mistral_chat(query,model,client,stream=False)
print()

In [11]:
print()
print("-" * 80)
print('Question: %s'%query)
print("-" * 80)
print("No streaming")
print()
print('Answer: ')
print(chat_response.choices[0].message.content)


--------------------------------------------------------------------------------
Question: What is the average grid for competitors with over 22 laps and time/retired of +17.276? Answer me as SQL and explain your solutions.
--------------------------------------------------------------------------------
No streaming

Answer: 
To provide you with an SQL query that could potentially answer your question, I'll have to make some assumptions about your database schema. Let's suppose you have a table named `races` with the following columns: `competitor_id`, `laps`, `time`, and `status`. The `status` column could have values like 'finished' or 'retired'.

Here's an SQL query that might work:

```sql
SELECT AVG(grid) as average_grid
FROM (
    SELECT competitor_id, grid
    FROM races
    WHERE laps > 22
    AND (time = '+17.276' OR status = 'retired')
) as subquery
JOIN competitors ON subquery.competitor_id = competitors.id;
```

This query first creates a subquery that selects all `competi

In [12]:
%mkdir /content/outputs/
import re
import time
import json

import numpy as np

from tqdm import tqdm
from datasets import load_dataset
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed

def main(tasks=TASKSTEST):
    openai.api_key = openai.api_key
    mmlu_prompt = json.load(open('/content/chain-of-thought-hub/MMLU/lib_prompt/mmlu-cot.json'))
    for task in tasks:

        print()
        print('Testing %s ...' % task)
        print()

        i = 0
        acc = 0
        task_data = load_dataset("lukaemon/mmlu", task, trust_remote_code=True)
        model="mistral-large-latest"

        with open('/content/outputs/test_%s_%s.txt' % (model, task), 'w') as fd:
        #with open('/content/outputs/test_gpt_3.5_turbo_%s.txt' % task, 'w') as fd:
            for q_ in tqdm(task_data['test'], total=len(task_data['test'])):
                q = q_['input'] + '\n'
                for letter in ['A', 'B', 'C', 'D']:
                    q += '(' + letter + ') ' + q_[letter] + ' '
                q += "\nA: Let's think step by step."

                prompt_q = mmlu_prompt[task] + "\n\n" + q
                #print(prompt_q)

                ### ADDED by Frank Morales 30/03/2023
                response=mistral_chat(prompt_q,model,client,stream=False)

                ### ADDED by Frank Morales 30/03/2023
                ans_model = response.choices[0].message.content
                #print(response.choices[0].message.content)

                ans_, residual = extract_ans(ans_model)

                a = q_['target']
                #print(a)
                fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
                i += 1

                if(test_answer_mmlu_(ans_, a)): acc += 1
            print('%s acc %.4f' % (task, acc / len(task_data['test'])))
        # write accuracy to file
        with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu'), 'a') as fd:
          fd.write('%s acc %.4f\n' % (task, acc / len(task_data['test'])))

    # write average accuracy to file
    acc_list = []
    #with open('/content/outputs/test_%s_%s_acc.txt' % (model, args.prompt_type), 'r') as fd2:

    # with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'multiple'), 'r') as fd2:
    with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu'), 'r') as fd2:
        for line in fd2:
            acc_list.append(float(line.split(' ')[2]))
    with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu'), 'a') as fd:
        fd.write('Average acc %.4f\n' % (np.mean(acc_list)))
    return

if __name__ == '__main__':
    main()


Testing college_computer_science ...



Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

100%|██████████| 100/100 [10:41<00:00,  6.42s/it]


...

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

100%|██████████| 112/112 [09:55<00:00,  5.31s/it]

machine_learning acc 0.5179


MODEL: mistral-large-latest

college_computer_science acc 0.6100
electrical_engineering acc 0.6069
machine_learning acc 0.5179
Average acc 0.5783

In [13]:
model="mistral-large-latest"

print()
print('MODEL: %s'%model)
print()
acc_file='/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu')
#print(acc_file)

with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu'), 'r') as fd:
     for line in fd:
            print(line)
            #acc_list.append(float(line.split(' ')[2]))


MODEL: mistral-large-latest

college_computer_science acc 0.6100

electrical_engineering acc 0.6069

machine_learning acc 0.5179

Average acc 0.5783

